# TP Partie 2 : Fine-tuning de LLM avec LoRA

## 📚 Contexte

Dans ce notebook, nous allons explorer le fine-tuning de modèles de langage avec la technique LoRA (Low-Rank Adaptation). Cette approche permet d'adapter des LLMs pour des tâches spécifiques.

## 🎯 Objectifs

1. **Comprendre** le fonctionnement de LoRA et ses avantages
2. **Préparer** le dataset synthétique généré précédemment
3. **Configurer** un entraînement LoRA avec Unsloth
4. **Entraîner** un modèle Qwen3-4B spécialisé en français
5. **Évaluer** les performances du modèle fine-tuné
6. **Publier** le modèle sur Hugging Face Hub

## 🔧 Architecture

Notre approche utilise :

1. **Unsloth** : Bibliothèque optimisée pour le fine-tuning LoRA
2. **Qwen3-4B** : Modèle de base pré-entraîné
3. **QLoRA** : Quantization + LoRA pour réduire la mémoire GPU
4. **Hugging Face Hub** : Publication et partage du modèle

## 2. Import des bibliothèques Unsloth

**Note importante** : Toutes les dépendances sont gérées via `uv sync` depuis le pyproject.toml.

### Installation préalable :
```bash
# Installation standard
uv sync

# Pour GPU CUDA support
uv sync --extra gpu

# Pour le développement
uv sync --extra dev
```

Si vous rencontrez des erreurs d'importation, vérifiez que vous avez bien exécuté la commande d'installation.

In [ ]:
# Import des bibliothèques Unsloth
try:
    from unsloth import FastLanguageModel
    print("✅ Unsloth importé avec succès!")
    
    # Vérification GPU
    import torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device utilisé: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name()}")
        print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("Veuillez exécuter 'uv sync' pour installer les dépendances.")
    print("Pour GPU support: uv sync --extra gpu")

## 2. Installation d'Unsloth

**Note importante** : Unsloth est une bibliothèque optimisée qui accélère significativement le fine-tuning LoRA.

In [ ]:
# Installation d'Unsloth (nécessite redémarrage du kernel)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

# Import après installation
from unsloth import FastLanguageModel

## 3. Chargement et préparation du dataset

Nous chargeons le dataset synthétique généré dans la partie précédente.

In [ ]:
# Configuration des chemins
data_dir = Path("../data/synthetic")

# Chargement du dataset
print("Chargement du dataset synthétique...")
if (data_dir / "synthetic_dataset_alpaca.json").exists():
    with open(data_dir / "synthetic_dataset_alpaca.json", 'r', encoding='utf-8') as f:
        alpaca_data = json.load(f)
    print(f"✅ Dataset chargé: {len(alpaca_data)} paires instruction-réponse")
else:
    print("❌ Dataset non trouvé. Veuillez exécuter le notebook 01 d'abord.")
    # Création d'un dataset d'exemple
    alpaca_data = [
        {
            "instruction": "Qu'est-ce que l'apprentissage automatique?",
            "input": "",
            "output": "L'apprentissage automatique est un sous-domaine de l'intelligence artificielle qui permet aux systèmes d'apprendre à partir de données sans être explicitement programmés."
        }
    ]

# Conversion en DataFrame pour analyse
df = pd.DataFrame(alpaca_data)
print(f"\n📊 Statistiques du dataset:")
print(f"   - Nombre total de paires: {len(df)}")
print(f"   - Longueur moyenne des instructions: {df['instruction'].str.len().mean():.0f} caractères")
print(f"   - Longueur moyenne des réponses: {df['output'].str.len().mean():.0f} caractères")

# Affichage des premières lignes
print("\n📝 Exemples de données:")
for i in range(min(3, len(alpaca_data))):
    print(f"\n--- Exemple {i+1} ---")
    print(f"Instruction: {alpaca_data[i]['instruction']}")
    print(f"Réponse: {alpaca_data[i]['output'][:100]}...")

## 4. Préparation des données pour l'entraînement

Nous formatons les données selon le format attendu par Unsloth pour le fine-tuning instructionnel.

In [ ]:
# Template de formatage pour le modèle
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = "<|im_end|>"  # Token de fin pour Qwen3

def format_prompts(examples):
    """
    Formate les exemples selon le template Alpaca pour le fine-tuning.
    """
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formatage du prompt
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

# Conversion en Dataset Hugging Face
dataset = Dataset.from_pandas(df)

# Application du formatage
print("Formatage des données pour l'entraînement...")
dataset = dataset.map(format_prompts, batched=True)

# Split train/validation
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

print(f"\n✅ Dataset préparé:")
print(f"   - Training set: {len(dataset_dict['train'])} exemples")
print(f"   - Validation set: {len(dataset_dict['validation'])} exemples")

# Affichage d'un exemple formaté
print("\n📝 Exemple formaté:")
print(dataset_dict['train'][0]['text'][:500] + "...")

## 5. Configuration du modèle et de LoRA

Nous configurons le modèle Qwen3-4B avec les paramètres LoRA pour un fine-tuning efficace.

In [ ]:
# Configuration du modèle
model_name = "Qwen/Qwen3-4B-Instruct-2507"
max_seq_length = 2048  # Longueur maximale de séquence
dtype = None  # Détection automatique du type
load_in_4bit = True  # Utilisation de la quantification 4-bit

print(f"Chargement du modèle {model_name}...")

# Chargement du modèle avec Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✅ Modèle chargé avec succès!")
print(f"   - Mémoire utilisée: {torch.cuda.memory_allocated() / 1e9:.1f} GB")
print(f"   - Mémoire réservée: {torch.cuda.memory_reserved() / 1e9:.1f} GB")

In [ ]:
# Configuration des paramètres LoRA
lora_config = {
    "r": 16,  # Rank des matrices de mise à jour
    "target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # Modules à adapter
    "lora_alpha": 32,  # Facteur d'échelle
    "lora_dropout": 0.05,  # Dropout pour régularisation
    "bias": "none",  # Pas d'adaptation des biais
    "use_gradient_checkpointing": True,  # Pour économiser la mémoire
    "random_state": 3407,
    "use_rslora": False,  # Ne pas utiliser scaled LoRA
    "loftq_config": None,  # Configuration LoftQ (pour quantification avancée)
}

print("Configuration LoRA:")
for key, value in lora_config.items():
    print(f"   - {key}: {value}")

# Application de la configuration LoRA
model = FastLanguageModel.get_peft_model(
    model,
    **lora_config
)

print("\n✅ Configuration LoRA appliquée!")
print(f"   - Paramètres entraînables: {model.print_trainable_parameters()}")

## 6. Configuration de l'entraînement

Nous configurons les hyperparamètres d'entraînement et le trainer.

In [ ]:
# Configuration des arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./lora_finetuned_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=3,  # Nombre d'époques
    learning_rate=2e-4,  # Taux d'apprentissage
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="./outputs",
    report_to="none",  # Désactiver wandb/tensorboard
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
)

print("Arguments d'entraînement:")
print(f"   - Batch size: {training_args.per_device_train_batch_size}")
print(f"   - Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   - Learning rate: {training_args.learning_rate}")
print(f"   - Epochs: {training_args.num_train_epochs}")
print(f"   - Warmup steps: {training_args.warmup_steps}")

In [ ]:
# Configuration du trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Ne pas packer les séquences
    args=training_args,
)

print("✅ Trainer configuré!")
print(f"   - Échantillons d'entraînement: {len(trainer.train_dataset)}")
print(f"   - Échantillons de validation: {len(trainer.eval_dataset)}")

## 7. Entraînement du modèle

**⚠️ Important** : Cette étape peut prendre du temps (15-30 minutes) selon votre GPU et la taille du dataset.

In [ ]:
# Démarrage de l'entraînement
print("🚀 Démarrage de l'entraînement LoRA...")
print(f"   - Dataset: {len(dataset_dict['train'])} exemples")
print(f"   - Paramètres LoRA: rank={lora_config['r']}, alpha={lora_config['lora_alpha']}")
print(f"   - Learning rate: {training_args.learning_rate}")
print(f"   - Epochs: {training_args.num_train_epochs}\n")

# Suivi de la mémoire avant entraînement
if torch.cuda.is_available():
    print(f"Mémoire GPU avant entraînement: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

# Lancement de l'entraînement
trainer_stats = trainer.train()

print("\n✅ Entraînement terminé!")
print(f"   - Perte finale: {trainer_stats.training_loss:.4f}")
print(f"   - Temps d'entraînement: {trainer_stats.metrics.get('train_runtime', 0):.1f} secondes")

# Suivi de la mémoire après entraînement
if torch.cuda.is_available():
    print(f"Mémoire GPU après entraînement: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

## 8. Évaluation du modèle

Nous évaluons les performances du modèle fine-tuné sur le jeu de validation.

In [ ]:
# Évaluation sur le jeu de validation
print("📊 Évaluation du modèle sur le jeu de validation...")
eval_results = trainer.evaluate()

print("\nRésultats de l'évaluation:")
for key, value in eval_results.items():
    if isinstance(value, (int, float)):
        print(f"   - {key}: {value:.4f}")

# Visualisation de la courbe d'apprentissage
if hasattr(trainer, "state") and trainer.state.log_history:
    log_history = trainer.state.log_history
    losses = [log.get('loss', 0) for log in log_history if 'loss' in log]
    steps = [log.get('step', 0) for log in log_history if 'loss' in log]
    
    plt.figure(figsize=(10, 6))
    plt.plot(steps, losses, 'b-', label='Training Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.title('Courbe d\'apprentissage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 9. Test d'inférence

Nous testons le modèle fine-tuné sur quelques exemples pour évaluer sa capacité à générer des réponses en français.

In [ ]:
# Configuration du modèle pour l'inférence
FastLanguageModel.for_inference(model)

# Fonction de génération
def generate_response(instruction: str, max_new_tokens: int = 512) -> str:
    """
    Génère une réponse à partir d'une instruction.
    """
    prompt = alpaca_prompt.format(
        instruction,
        "",  # Input vide
        ""  # Laisser vide pour la génération
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # Extraire seulement la réponse générée
    response = response.split("### Response:")[1].strip()
    return response

# Test avec quelques exemples
test_instructions = [
    "Qu'est-ce que l'intelligence artificielle?",
    "Explique le principe du fine-tuning.",
    "Quels sont les avantages de LoRA?"
]

print("🧪 Tests d'inférence:\n")
for i, instruction in enumerate(test_instructions):
    print(f"--- Test {i+1} ---")
    print(f"Instruction: {instruction}")
    response = generate_response(instruction)
    print(f"Réponse: {response[:300]}...\n")

## 10. Analyse des compromis LoRA

Nous analysons les avantages et inconvénients de l'approche LoRA.

In [ ]:
# Analyse des paramètres entraînables
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
percentage = (trainable_params / total_params) * 100

print("📊 Analyse des paramètres:")
print(f"   - Paramètres totaux: {total_params:,}")
print(f"   - Paramètres entraînables: {trainable_params:,}")
print(f"   - Pourcentage entraînable: {percentage:.2f}%")

# Comparaison des approches
comparison_data = {
    "Approche": ["Full Fine-tuning", "LoRA (r=16)", "LoRA (r=8)", "LoRA (r=32)"],
    "Paramètres entraînables": [
        f"{total_params:,}",
        f"{trainable_params:,}",
        f"{trainable_params//2:,}",
        f"{trainable_params*2:,}"
    ],
    "Mémoire GPU estimée": ["~24GB", "~8GB", "~6GB", "~10GB"],
    "Temps d'entraînement": ["100%", "~30%", "~20%", "~40%"],
    "Performance relative": ["100%", "~95%", "~85%", "~98%"]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n🔄 Comparaison des approches:")
print(comparison_df.to_string(index=False))

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des types de paramètres
param_types = ['Entraînables', 'Gelés']
param_counts = [trainable_params, total_params - trainable_params]

axes[0].pie(param_counts, labels=param_types, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribution des paramètres')

# Efficacité mémoire
approaches = ['Full FT', 'LoRA r=16', 'LoRA r=8', 'LoRA r=32']
memory_usage = [100, 33, 25, 42]  # Pourcentage du full fine-tuning

axes[1].bar(approaches, memory_usage, color=['red', 'green', 'blue', 'orange'])
axes[1].set_ylabel('Mémoire GPU relative (%)')
axes[1].set_title('Efficacité mémoire')
axes[1].set_ylim(0, 120)

# Ajout des valeurs sur les barres
for i, v in enumerate(memory_usage):
    axes[1].text(i, v + 2, f'{v}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 11. Sauvegarde du modèle

Nous sauvegardons le modèle fine-tuné pour une utilisation future.

In [ ]:
# Création du dossier de sauvegarde
model_save_path = Path("../models/lora_qwen3_french")
model_save_path.mkdir(parents=True, exist_ok=True)

print(f"💾 Sauvegarde du modèle dans {model_save_path}...")

# Sauvegarde du modèle LoRA
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Sauvegarde de la configuration
config = {
    "base_model": model_name,
    "lora_config": lora_config,
    "training_args": {
        "num_train_epochs": training_args.num_train_epochs,
        "learning_rate": training_args.learning_rate,
        "per_device_train_batch_size": training_args.per_device_train_batch_size,
    },
    "dataset_info": {
        "train_size": len(dataset_dict['train']),
        "val_size": len(dataset_dict['validation']),
        "source": "synthetic_dataset"
    },
    "results": {
        "final_loss": trainer_stats.training_loss,
        "eval_results": eval_results
    }
}

with open(model_save_path / "config.json", 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f"✅ Modèle sauvegardé avec succès!")
print(f"   - Poids du modèle: {sum(os.path.getsize(model_save_path/f) for f in os.listdir(model_save_path) if os.path.isfile(model_save_path/f)) / 1e6:.1f} MB")

# Optionnel: Sauvegarde en 16-bit pour l'inférence
print("\n💾 Sauvegarde en 16-bit pour l'inférence...")
model.save_pretrained_merged(
    model_save_path / "merged_16bit",
    tokenizer,
    save_method="merged_16bit"
)
print("✅ Sauvegarde 16-bit terminée!")

## 12. Préparation pour la publication sur Hugging Face

Nous préparons le modèle pour sa publication sur Hugging Face Hub.

In [ ]:
# Création du README pour Hugging Face
readme_content = f"""---
library_name: transformers
license: mit
base_model: {model_name}
tags:
- trl
- sft
- generated_from_trainer
- french
- lora
- qwen3
model-index:
- name: Qwen3-4B-French-LoRA
  results: []
---

# Qwen3-4B-French-LoRA

## Description

Ce modèle est une version fine-tunée de Qwen3-4B-Instruct spécialisée pour le français. Il a été entraîné sur un dataset synthétique d'instructions-réponses en français.

## Détails d'entraînement

- **Base Model**: {model_name}
- **Training Framework**: Unsloth + TRL
- **Quantization**: 4-bit (QLoRA)
- **LoRA Rank**: {lora_config['r']}
- **LoRA Alpha**: {lora_config['lora_alpha']}
- **Training Epochs**: {training_args.num_train_epochs}
- **Learning Rate**: {training_args.learning_rate}
- **Dataset Size**: {len(dataset_dict['train'])} examples

## Performance

- **Training Loss**: {trainer_stats.training_loss:.4f}
- **Validation Loss**: {eval_results.get('eval_loss', 'N/A')}

## Utilisation

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Chargement du modèle
model_path = "votre-username/Qwen3-4B-French-LoRA"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Formatage du prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{{}}

### Input:
{{}}

### Response:
{{}}"""

# Génération
prompt = alpaca_prompt.format(
    "Qu'est-ce que l'intelligence artificielle?",
    "",
    ""
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
```

## Limitations

- Le modèle est spécialisé pour le français
- Performances limitées pour les tâches nécessitant un raisonnement complexe
- Peut générer des informations incorrectes (hallucinations)

## Éthique

Ce modèle est destiné à des fins éducatives et de recherche. Les utilisateurs doivent être conscients des biais potentiels et utiliser le modèle de manière responsable.
"""

# Sauvegarde du README
with open(model_save_path / "README.md", 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("📝 README créé pour Hugging Face Hub")

# Instructions pour la publication
print("\n🚀 Instructions pour la publication sur Hugging Face Hub:")
print("1. Créez un compte sur Hugging Face: https://huggingface.co/join")
print("2. Installez l'CLI Hugging Face: pip install 'huggingface_hub[cli]'")
print("3. Connectez-vous: huggingface-cli login")
print("4. Créez un nouveau repository: huggingface-cli repo create Qwen3-4B-French-LoRA --type model")
print("5. Uploadez votre modèle: huggingface-cli upload ./models/lora_qwen3_french votre-username/Qwen3-4B-French-LoRA")

# Script de publication automatisé
publish_script = f"""#!/bin/bash
# Script pour publier le modèle sur Hugging Face Hub

# Remplacez VOTRE_USERNAME par votre username Hugging Face
USERNAME="VOTRE_USERNAME"
MODEL_NAME="Qwen3-4B-French-LoRA"
MODEL_PATH="./models/lora_qwen3_french"

echo "Publication du modèle $MODEL_NAME sur Hugging Face Hub..."

# Vérification que le modèle existe
if [ ! -d "$MODEL_PATH" ]; then
    echo "Erreur: Le modèle n'existe pas dans $MODEL_PATH"
    exit 1
fi

# Upload du modèle
huggingface-cli upload $MODEL_PATH $USERNAME/$MODEL_NAME \
    --repo-type model \
    --private  # Changez à --public pour un modèle public

echo "✅ Modèle publié avec succès!"
echo "🔗 Visitez: https://huggingface.co/$USERNAME/$MODEL_NAME"
"""

with open(model_save_path / "publish.sh", 'w') as f:
    f.write(publish_script)

# Rendre le script exécutable
os.chmod(model_save_path / "publish.sh", 0o755)
print("\n📜 Script de publication créé: publish.sh")

## 13. Résumé et conclusions

In [ ]:
# Résumé final
print("🎉 TP de fine-tuning LoRA terminé!\n")
print("📊 Résumé des résultats:")
print(f"   - Modèle de base: {model_name}")
print(f"   - Paramètres LoRA: rank={lora_config['r']}, alpha={lora_config['lora_alpha']}")
print(f"   - Dataset d'entraînement: {len(dataset_dict['train'])} exemples")
print(f"   - Époques d'entraînement: {training_args.num_train_epochs}")
print(f"   - Perte finale: {trainer_stats.training_loss:.4f}")
print(f"   - Perte validation: {eval_results.get('eval_loss', 'N/A')}")
print(f"   - Mémoire GPU utilisée: {torch.cuda.memory_allocated() / 1e9:.1f} GB")

print("\n📁 Fichiers créés:")
print(f"   - {model_save_path.name}/ (modèle LoRA)")
print(f"   - {model_save_path.name}/merged_16bit/ (modèle fusionné)")
print(f"   - {model_save_path.name}/README.md (documentation)")
print(f"   - {model_save_path.name}/publish.sh (script de publication)")

print("\n🔍 Points clés appris:")
print("   1. LoRA permet de fine-tuner avec ~1% des paramètres")
print("   2. QLoRA réduit significativement la mémoire GPU")
print("   3. Unsloth accélère l'entraînement de 2-3x")
print("   4. Le modèle reste petit et facile à partager")

print("\n🚀 Prochaines étapes:")
print("   1. Publier le modèle sur Hugging Face Hub")
print("   2. Tester avec des données réelles")
print("   3. Expérimenter avec différents paramètres LoRA")
print("   4. Essayer d'autres techniques de fine-tuning")

## 14. Pour aller plus loin

### Expérimentations suggérées

1. **Variation du rank LoRA**: Tester r=8, r=16, r=32 et comparer les performances
2. **Différents taux d'apprentissage**: 1e-4, 2e-4, 5e-4
3. **Augmentation du dataset**: Générer plus de données synthétiques
4. **Techniques avancées**:
   - DoRA (Weight-Decomposed LoRA)
   - VeRA (Vector-based Random Matrix Adaptation)
   - QLoRA avec différents niveaux de quantification

### Optimisations possibles

1. **Batch processing**: Augmenter la taille du batch si mémoire permet
2. **Gradient checkpointing**: Activé par défaut dans Unsloth
3. **Mixed precision**: Utiliser bf16 si disponible
4. **Data parallelism**: Pour multi-GPU

### Évaluation approfondie

1. **Benchmarks automatiques**: Utiliser des benchmarks français
2. **Évaluation humaine**: Qualité des réponses générées
3. **Comparaison baseline**: Contre le modèle de base
4. **Analyse d'erreurs**: Comprendre les faiblesses du modèle

### Déploiement

1. **API REST**: Créer une API pour le modèle
2. **Optimisation inférence**: vLLM, TensorRT-LLM
3. **Quantification post-entraînement**: GGUF, AWQ
4. **Monitoring**: Suivi des performances en production